Constantes: tamanho do Mock e instância do Faker


In [ ]:
from faker import Faker
USER_COUNT = 118758
fake = Faker()

# Funções usadas ao longo do Notebook

Gerar dia válido dentro de um determinado mês. Prioriza primeiros dias.

In [ ]:
import numpy as np
import calendar
from scipy.stats import expon

def generate_days(
        month: tuple[int, int],
        size: int,
        start_day: int=1
    ) -> np.ndarray | None:
    _, last_day = calendar.monthrange(*month)
    if start_day == last_day:
        return np.full((size,), start_day)
    elif start_day > last_day:
        return None
    available_days = np.arange(start=start_day, stop=last_day + 1)
    weights = np.copy(available_days)
    weights = weights / np.sum(weights)
    weights = expon.pdf(weights)
    weights = weights / np.sum(weights)
    return np.random.choice(available_days, size=size, p=weights)

# Dados de Usuário


Iremos gerar um DataFrame Pandas para conter informações de usuário, assim como o endereço do mesmo.


In [ ]:
import pandas as pd

# Utilizando index do pandas como id
users = pd.DataFrame({
    'created_at': [],
    'birthday': [],
    'city': [],
    'state': [],
    'country': []
})
display(users)

Gerando _created_at_ (omitindo _updated_at_ pois ele não aparece em nenhum lugar do relatório)


Utilizando distribuição ponderada, priorizando os meses 2, 3 e 4.


In [ ]:
from scipy.stats import skewnorm
from matplotlib import pyplot as plt
from datetime import datetime, timedelta

# Dia 1 -> 01/01/2023
# Dia 2 -> 02/01/2023
# ...
# Dia 181 -> 30/06/2023
# Priorizar meses 2, 3 e 4:
# Mês 1: do dia 1 ao 31
# Mês 2: do dia 32 ao 59
# Mês 3: do dia 60 ao 90
# Mês 4: do dia 91 ao 120
# Mês 5: do dia 121 ao 151
# Mês 6: dia 152 ao 181
# Média: dia 76
# Desvio padrão: 30
# Skewness: 0.5
first_day = 1
last_day = 181
print(f'Primeiro dia: {datetime(2023, 1, 1) + timedelta(days=first_day - 1)}')
print(f'Último dia: {datetime(2023, 1, 1) + timedelta(days=last_day - 1)}')
available_days = np.arange(first_day, last_day + 1)
lin_range = np.linspace(0, 1, last_day - first_day + 1)
weights = skewnorm.pdf(x=lin_range, a=3.0, loc=0.3, scale=0.75)
# Noise
weights = np.abs(weights + np.random.normal(loc=0, scale=0.01, size=last_day - first_day + 1))
weights = weights / np.sum(weights)
plt.gca().vlines(np.array([32, 122]), ymin=0, ymax=np.max(weights), color='C1')
plt.plot(np.arange(first_day, last_day + 1), weights)
plt.show()

days = np.random.choice(available_days, size=USER_COUNT, p=weights)
plt.hist(days, bins=last_day - first_day + 1)
plt.show()

Horas não são aleatórias: priorizar horários de pico


In [ ]:
import numpy as np
from scipy.stats import skewnorm


def gen_random_minutes(size):
    minutes = skewnorm.rvs(a=-0.5, loc=960, scale=240, size=size)
    while True:
        minutes = np.round(minutes, 0).astype(int)
        minutes_outside_interval = np.argwhere(
            (minutes < 0) | (minutes >= 1440)).flatten()
        if (minutes_outside_interval.size == 0):
            break
        minutes[minutes_outside_interval] = skewnorm.rvs(
            a=-0.5, loc=960, scale=240, size=len(minutes_outside_interval))
    return minutes


minutes = gen_random_minutes(USER_COUNT)
hours = minutes // 60
minutes = minutes % 60

Adicionando created_at


In [ ]:
from datetime import datetime, timedelta
import numpy as np

def _created_at_gen(day, hour, minute):
    return (datetime(
        2023,
        1,
        1,
        hour,
        minute,
        np.random.randint(0, 60)) + timedelta(days=int(day - 1))).isoformat()
created_at_gen = np.vectorize(_created_at_gen)
created_at = created_at_gen(days, hours, minutes)
users['created_at'] = created_at

Gerando datas de nascimento


Ano será escolhido através de uma distribuição que prioriza fim dos anos 90 e anos 2000


In [ ]:
from matplotlib import pyplot as plt
from scipy.stats import alpha

available_years = np.arange(2005, 1939, -1)
lins = np.linspace(0, 1, len(available_years))
weights = alpha.pdf(lins, 3.6, loc=-0.4, scale=2)
weights = weights / np.sum(weights)
plt.plot(available_years, weights)
plt.show()
plt.gca().vlines([1990, 1995, 2000, 2005], ymin=0, ymax=np.max(weights), color='C1')
plt.gca().hlines(0, xmin=1939, xmax=2005, color='C1')
random_years = np.random.choice(available_years, size=USER_COUNT, p=weights)
plt.hist(random_years, bins=available_years.size)
plt.show()

Mês e dia são completamente randômicos, utiliza distribuição uniforme


In [ ]:
random_dates_with_year = np.empty((USER_COUNT,), dtype=object)
for i in range(USER_COUNT):
    year = random_years[i]
    lower_bound = datetime(year, 1, 1)
    upper_bound = datetime(year, 12, 31)
    random_date = fake.date_between(
        start_date=lower_bound, end_date=upper_bound).strftime('%Y-%m-%d')
    random_dates_with_year[i] = random_date
users['birthday'] = random_dates_with_year

Ordenando DataFrame de forma que created_at seja crescente


In [ ]:
users = users.sort_values(by='created_at')
users = users.reset_index(drop=True)
display(users)

Obtendo índices para cada mês

In [ ]:
users['c_at_month'] = users['created_at'].str.slice(start=5, stop=7).apply(lambda str: int(str))
month_intervals = []
for i in range(1, 7):
    idxs = users.index[users['c_at_month'] == i].tolist()
    start_idx = idxs[0]
    end_idx = idxs[-1]
    month_intervals.append((start_idx, end_idx))

# Endereços


Queremos mudar a distribuição de endereços com base na data do onboarding. Portanto, é interessante antes criar uma função que recebe alguns parâmetros, como:

- Importância da população para decidir a cidade
- Presença de cidades fora do Brasil?
- etc


## Cidades do Brasil

Utilizando .csv de municipios contidos [aqui](http://blog.mds.gov.br/redesuas/wp-content/uploads/2018/06/Lista_Munic%C3%ADpios_com_IBGE_Brasil_Versao_CSV.csv)


In [ ]:
# Renomeie o CSV
municipios = pd.read_csv('municipios.csv', sep=';')
# Ignorando duas colunas
municipios = municipios.drop(columns=[
    'ConcatUF+Mun',
    'Unnamed: 9']).sort_values(by=['População 2010'], ascending=False)
# Ignorando munícipios sem informação de população
municipios = municipios.dropna(subset=['População 2010'])
municipios = municipios.reset_index(drop=True)
display(municipios.head(5))
# Transformando Presidente Prudente em uma cidade com 2M de habitantes
# Só pelo lulz
municipios.loc[
    municipios['Município'] == 'Presidente Prudente',
    'População 2010'
  ] = 2000000
display(municipios[municipios['Município'] == 'Presidente Prudente'])

## Cidades fora do Brasil

Utilizando csv que pode ser encontrado [aqui](https://simplemaps.com/static/data/world-cities/basic/simplemaps_worldcities_basicv1.76.zip)


Países da América Latina


In [ ]:
latin_america_countries = [
    'Argentina',
    'Bolivia',
    'Chile',
    'Colombia',
    'Ecuador',
    'Paraguay',
    'Peru',
    'Uruguay'
]

In [ ]:
world_cities = pd.read_csv('worldcities.csv')
# Ignorando cidades sem dados populacionais
world_cities = world_cities.dropna(subset=['population'])
# Incluindo apenas cidades da América Latina (exclui Brasil)
world_cities = world_cities[world_cities['country'].isin(latin_america_countries)]
# Excluindo cidades com população menor que 500 mil habitantes
world_cities = world_cities[world_cities['population'] >= 500000]
world_cities = world_cities.reset_index(drop=True)
display(world_cities.head(5))

Finalmente, preparando a função


In [ ]:
def generate_random_cities(size, population_importance=1.6, non_brazilian_percent=0.0):
    brazilian_percent = 1.0 - non_brazilian_percent
    brazilian_size = int(size * brazilian_percent)
    non_brazilian_size = size - brazilian_size
    cities = pd.DataFrame(columns=['city', 'state', 'country'])
    # Cidades brasileiras
    brazilian_cities = pd.DataFrame(columns=['city', 'state', 'country'])
    brazilian_population_power = np.power(
        municipios['População 2010'], population_importance)
    brazilian_population_weights = brazilian_population_power / \
        np.sum(brazilian_population_power)
    brazilian_cities_idx = np.random.choice(
        municipios.index, p=brazilian_population_weights, size=brazilian_size)
    brazilian_cities['city'] = municipios.loc[brazilian_cities_idx,
                                              'Município'].values
    brazilian_cities['state'] = municipios.loc[brazilian_cities_idx, 'UF'].values
    brazilian_cities['country'] = 'Brazil'
    # Cidades não brasileiras
    non_brazilian_cities = pd.DataFrame(columns=['city', 'state', 'country'])
    non_brazilian_population_power = np.power(
        world_cities['population'], population_importance)
    non_brazilian_population_weights = non_brazilian_population_power / \
        np.sum(non_brazilian_population_power)
    non_brazilian_cities_idx = np.random.choice(
        world_cities.index, p=non_brazilian_population_weights, size=non_brazilian_size)
    non_brazilian_cities['city'] = world_cities.loc[non_brazilian_cities_idx, 'city'].values
    non_brazilian_cities['state'] = ''
    non_brazilian_cities['country'] = world_cities.loc[non_brazilian_cities_idx, 'country'].values
    # Concatenando
    cities = pd.concat([brazilian_cities, non_brazilian_cities])
    cities = cities.reset_index(drop=True)
    return cities

Primeiros 2 meses: focar em capitais do Brasil


In [ ]:
start_idx = month_intervals[0][0]
end_idx = month_intervals[1][1]
count = end_idx - start_idx
cities = generate_random_cities(count, population_importance=2.5)
users.loc[users.index[start_idx: end_idx], [
    'city', 'state', 'country']] = cities.values

2 próximos meses: cidades brasileiras com menor população


In [ ]:
start_idx = month_intervals[2][0]
end_idx = month_intervals[3][1]
count = end_idx - start_idx
cities = generate_random_cities(count, population_importance=1.5)
users.loc[users.index[start_idx: end_idx], [
    'city', 'state', 'country']] = cities.values

2 últimos meses: cidades no exterior


In [ ]:
start_idx = month_intervals[4][0]
end_idx = month_intervals[5][1]
count = end_idx - start_idx
cities = generate_random_cities(
    count, population_importance=1.0, non_brazilian_percent=0.05)
users.loc[users.index[start_idx: end_idx], [
    'city', 'state', 'country']] = cities.values

Resultado final


In [ ]:
display(users['country'].value_counts())
display(users['state'].value_counts())
display(users['city'].value_counts())

# Salvando dados de usuário em CSV


In [ ]:
from datetime import datetime

def age_from_birthday(birthday_str):
    birthday = datetime.fromisoformat(birthday_str)
    difference = datetime.now() - birthday
    age = difference.days // 365
    return age


users['age'] = users['birthday'].apply(age_from_birthday)
display(users)
users.drop('c_at_month', axis=1).to_csv('users.csv', index_label='id')

# Transferências


## Função que gera transferências com determinada distribuição de valores e horas


In [ ]:
def generate_transfers(
        user_ids,
        transfer_quantity,
        loc=50,
        scale=25,
        num_outliers=0
    ):
    transfers = pd.DataFrame(
        columns=[
            'id_from',
            'id_to',
            'hour',
            'minute',
            'scheduled_date',
            'value',
            'status'
        ]
    )
    ids_from, ids_to = user_ids
    total_size = transfer_quantity
    values = np.random.normal(
        loc=loc,
        scale=scale,
        size=total_size - num_outliers
    )
    values = np.append(values, np.random.normal(
            loc=loc*10,
            scale=loc*10,
            size=num_outliers
        )
    )
    values = np.abs(values)
    values = np.round(values, 2)
    status = np.random.choice(
        ['FAILED', 'DONE', 'SCHEDULED'],
        p=[0.002, 0.75, 0.248], size=total_size)
    # Chose random ids
    transfers_ids_from = np.random.choice(ids_from, size=total_size)
    transfers_ids_to = np.random.choice(ids_to, size=total_size)
    transfers['id_from'] = transfers_ids_from
    transfers['id_to'] = transfers_ids_to
    transfers['value'] = values
    transfers['status'] = status
    # Horário das transferências
    minutes = gen_random_minutes(total_size)
    hours = minutes // 60
    minutes = minutes % 60
    transfers['hour'] = hours
    transfers['minute'] = minutes
    return transfers


def decide_transfers_dates(transfers: pd.DataFrame):
    users_created_at = users[['created_at']]
    transfer_with_user_info = transfers\
        .join(users_created_at, on='id_from')\
        .rename(columns={'created_at': 'created_at_x'})
    transfer_with_user_info = transfer_with_user_info\
        .join(users_created_at, on='id_to')\
        .rename(columns={'created_at': 'created_at_y'})
    transfer_with_user_info['lower_bound'] = transfer_with_user_info\
        .apply(
            lambda row: max(
                row['created_at_x'],
                row['created_at_y']
            ),
            axis=1
    )
    transfer_with_user_info = transfer_with_user_info\
        .drop(
            ['created_at_x', 'created_at_y'],
            axis=1
        )
    def gen_month_day(hour, minute, lower_bound):
        lower_bound = datetime.fromisoformat(lower_bound)
        month = np.random.randint(lower_bound.month, 8)
        _, last_day = calendar.monthrange(2023, month)
        if month == 7:
            day = 1
        else:
            if month > lower_bound.month:
                start_day = 1
            else:
                start_day = lower_bound.day
            date = datetime(2023, month, start_day, int(hour), int(minute))
            if (date <= lower_bound):
                start_day += 1
                if start_day > last_day:
                    month += 1
                    start_day = 1
            day = generate_days((2023, month), 1, start_day=start_day)[0]
        date = datetime(2023, month, day, hour, minute)
        if (date > lower_bound):
            return (day, month)
    gen_month_day_vectorized = np.vectorize(gen_month_day)
    temp = np.array([
        transfer_with_user_info['hour'],
        transfer_with_user_info['minute'],
        transfer_with_user_info['lower_bound']
    ])
    day, month = gen_month_day_vectorized(*temp)
    temp_t = temp.transpose()
    temp = np.array([
        month,
        day,
        temp_t[:, 0],
        temp_t[:, 1]
    ])
    def gen_date_time(month, day, hour, minute):
        return datetime(
            2023,
            month,
            day,
            hour,
            minute,
            np.random.randint(0, 60)).isoformat()
    gen_date_time_vectorized = np.vectorize(gen_date_time)
    date_times = gen_date_time_vectorized(*temp)
    new_transfers = transfers.copy()
    new_transfers.drop(['hour', 'minute'], axis=1, inplace=True)
    new_transfers['time'] = date_times
    return new_transfers

In [ ]:
def add_user_info(transfers: pd.DataFrame):
    users_info = users[['created_at', 'city', 'state', 'country', 'age']]
    transfer_joined = transfers.join(users_info, on='id_from')
    transfer_joined = transfer_joined.join(users_info, on='id_to', lsuffix="_from", rsuffix="_to")
    return transfer_joined

Usuários na faixa de 18-25: valores mais baixos


In [ ]:
user_subpopulation = users['age'].between(18, 25)
subpopulation_size = user_subpopulation.count()
active_sample_size = int(0.75 * subpopulation_size)
active_users_ids = np.random.choice(users[user_subpopulation].index, size=active_sample_size)
transfers_amount = 5 * active_sample_size
transfers_18_25 = generate_transfers(
    (active_users_ids, users.index), transfers_amount, loc=80, scale=50, num_outliers=int(transfers_amount * 0.01))

In [ ]:
transfers_18_25

In [ ]:
transfers_18_25 = decide_transfers_dates(transfers_18_25)
display(transfers_18_25)

In [ ]:
from datetime import datetime

def generate_scheduled_date(status, time):
    if status == 'SCHEDULED':
        return fake.date_between(
            datetime.fromisoformat(time),
            datetime(2023, 7, 31))
    else:
        return None

generate_scheduled_date_vectorized = np.vectorize(generate_scheduled_date)
input_matrix = np.array([
    transfers_18_25['status'],
    transfers_18_25['time']
])
scheduled = generate_scheduled_date_vectorized(*input_matrix)
transfers_18_25['scheduled_date'] = scheduled
display(transfers_18_25)

In [ ]:
transfers = pd.concat([transfers_18_25], axis=0)

In [ ]:
display(transfers)

Usuários na faixa de 26-60: valores mais altos


In [ ]:
user_subpopulation = users['age'].between(26, 60)
subpopulation_size = user_subpopulation.count()
active_sample_size = int(0.87 * subpopulation_size)
active_users_ids = np.random.choice(users[user_subpopulation].index, size=active_sample_size)
transfers_amount = 5 * active_sample_size
transfers_26_60 = generate_transfers(
    (active_users_ids, users.index), transfers_amount, loc=500, scale=200, num_outliers=int(transfers_amount * 0.01))
transfers_26_60 = decide_transfers_dates(transfers_26_60)
generate_scheduled_date_vectorized = np.vectorize(generate_scheduled_date)
input_matrix = np.array([
    transfers_26_60['status'],
    transfers_26_60['time']
])
scheduled = generate_scheduled_date_vectorized(*input_matrix)
transfers_26_60['scheduled_date'] = scheduled

In [ ]:
transfers = pd.concat([transfers, transfers_26_60], axis=0)
transfers = transfers.reset_index()
transfers = transfers.drop('index', axis=1)
display(transfers)

Usuários na faixa de 60+: valores mais baixos


In [ ]:
user_subpopulation = users['age'].between(61, 999)
subpopulation_size = user_subpopulation.count()
active_sample_size = int(0.5 * subpopulation_size)
active_users_ids = np.random.choice(users[user_subpopulation].index, size=active_sample_size)
transfers_amount = 3 * active_sample_size
transfers_61_plus = generate_transfers(
    (active_users_ids, users.index), transfers_amount, loc=200, scale=80, num_outliers=int(transfers_amount * 0.001))
transfers_61_plus = decide_transfers_dates(transfers_61_plus)
generate_scheduled_date_vectorized = np.vectorize(generate_scheduled_date)
input_matrix = np.array([
    transfers_61_plus['status'],
    transfers_61_plus['time']
])
scheduled = generate_scheduled_date_vectorized(*input_matrix)
transfers_61_plus['scheduled_date'] = scheduled

In [ ]:
transfers = pd.concat([transfers, transfers_61_plus], axis=0)
transfers = transfers.reset_index()
transfers = transfers.drop('index', axis=1)
display(transfers)

In [ ]:
# Ordenando por `time`
transfers = transfers.sort_values(by='time')
transfers = transfers.reset_index()
transfers = transfers.drop('index', axis=1)
display(transfers)

Removendo transferências após o dia 20/07/2023

In [ ]:
tranfers_time = transfers['time']
transfer_time_as_datetime = np.vectorize(datetime.fromisoformat)(tranfers_time)
check = np.vectorize(lambda dt: dt > datetime(2023, 7, 20))
dropped_idx = np.argwhere(check(transfer_time_as_datetime)).flatten()
transfers = transfers.drop(index=dropped_idx)
display(transfers)

Adicionando informação de usuários (evitando join no Looker Studio)

In [ ]:
transfers = add_user_info(transfers)
display(transfers)

In [ ]:
import pandas as pd

transfers = pd.read_csv("transfers.csv")

In [ ]:
transfers_eights = np.array_split(transfers, 8)
for i, df in enumerate(transfers_eights):
    df.to_csv(f'transfers_{i}.csv')

In [ ]:
# Sample (60%)
transfers = transfers.sample(frac=0.88)
transfers = transfers.reset_index()
transfers = transfers.drop("index", axis=1)
display(transfers)

In [ ]:
transfers.to_csv("transfers.csv", index=False)

# todo: login e atividades